In [ ]:
from sklearn import datasets as ds  
from sklearn.model_selection import train_test_split
import numpy as np
import math
from numpy import random
import matplotlib.pyplot as plt

def Loss(x_train,y_train,w,lmd):
    temp1=0
    n=0
    for x_sample, y_sample in zip(x_train,y_train):
        s=-y_sample*np.dot(x_sample,w)
        temp1+=np.log(1+np.exp(0))
        n+=1
    loss=temp1/n+lmd*np.dot(w.T,w)/2
    return loss

def Grad(x_train,y_train,w,lmd):
    temp2=0
    n=0
    for x_sample, y_sample in zip(x_train,y_train):
        temp2+=y_sample*x_sample/(1 + np.exp(np.dot(y_sample, np.dot(x_sample, w))))
        n+=1
    grad=-temp2/n+lmd*w
    return grad

def NAG(x_train,y_train,w,lmd,v,h):  
    g=Grad(x_train,y_train,w,lmd)
    v=lmd*v+h*g
    w=w-v
    return w
    
def RMSProp(x_train,y_train,w,lmd,e,h,G):
    g=Grad(x_train,y_train,w,lmd)
    G=lmd*G+np.dpt(1-lmd,np.dot(g,g))
    w=w-np.dot(h/np.sqrt(G+e),g)
    return w

def AdaDelta(x_train,y_train,w,lmd,e,dt,G):
    g=Grad(x_t,y_t,w,lmd)
    G=lmd*G+np.dpt(1-lmd,np.dot(g,g))
    dw=-np.dot(np.sqrt(dt+e)/np.sqrt(G+e),g)
    w=w+dw
    dt=lmd*dt+np.dot(1-lmd,np.dot(dw,dw))
    return w

def Adam(x_train,y_train,w,lmd,b,m,e,h,G):
    g=Grad(x_t,y_t,w,lmd)
    m=b*m+(1-b)*g
    G=lmd*G+np.dpt(1-lmd,np.dot(g,g))
    a1=h*(np.sqrt(1-lmd)/(1-b))
    lmd*=lmd
    b*=b
    w=w-a1*(m/np.sqrt(G+e))
    return w

def iterationNAG(x_train,x_validation,y_train,y_validation,w,lmd,v,h):
    itera = 100
    lr = 0.0001
    train_loss=[]
    validation_loss=[]
    for i in range(itera):
        loss_t=Loss(x_train,y_train,w,lmd)
        tx,ty=x_train.shape
        lt=loss_t[0,0]/tx
        train_loss.append(lt)
        loss_v=Loss(x_validation,y_validation,w,lmd)
        vx,vy=x_validation.shape
        lv=loss_v[0,0]/vx
        validation_loss.append(lv)
        w=NAG(x_train,y_train,w,lmd,v,h)
    return w,train_loss,validation_loss

def iterationRMSProp(x_train,x_validation,y_train,y_validation,w,lmd,e,h,G):
    itera = 100
    lr = 0.0001
    train_loss=[]
    validation_loss=[]
    for i in range(itera):
        loss_t=Loss(x_train,y_train,w,lmd)
        tx,ty=x_train.shape
        lt=loss_t[0,0]/tx
        train_loss.append(lt)
        loss_v=Loss(x_validation,y_validation,w,lmd)
        vx,vy=x_validation.shape
        lv=loss_v[0,0]/vx
        validation_loss.append(lv)
        w=RMSProp(x_train,y_train,w,lmd,e,h,G)
    return w,train_loss,validation_loss

def iterationAdaDelta(x_train,x_validation,y_train,y_validation,w,lmd,e,dt,G):
    itera = 100
    lr = 0.0001
    train_loss=[]
    validation_loss=[]
    for i in range(itera):
        loss_t=Loss(x_train,y_train,w,lmd)
        tx,ty=x_train.shape
        lt=loss_t[0,0]/tx
        train_loss.append(lt)
        loss_v=Loss(x_validation,y_validation,w,lmd)
        vx,vy=x_validation.shape
        lv=loss_v[0,0]/vx
        validation_loss.append(lv)
        w=AdaDelta(x_train,y_train,w,lmd,e,dt,G)
    return w,train_loss,validation_loss

def iterationAdam(x_train,x_validation,y_train,y_validation,w,lmd,b,m,e,h,G):
    itera = 100
    lr = 0.0001
    train_loss=[]
    validation_loss=[]
    for i in range(itera):
        loss_t=Loss(x_train,y_train,w,lmd)
        tx,ty=x_train.shape
        lt=loss_t[0,0]/tx
        train_loss.append(lt)
        loss_v=Loss(x_validation,y_validation,w,lmd)
        vx,vy=x_validation.shape
        lv=loss_v[0,0]/vx
        validation_loss.append(lv)
        w=Adam(x_train,y_train,w,lmd,b,m,e,h,G)
    return w,train_loss,validation_loss

if __name__ == '__main__':
    x_train, y_train = ds.load_svmlight_file('E:/test/train.txt')
    x_validation, y_validation = ds.load_svmlight_file('E:/test/val.txt')
    x_train = x_train.toarray()
    x_t=[]
    y_t=[]
    yl=len(y_train)
    y_train=y_train.reshape(yl,1)
    n_sample,n_feature=x_train.shape
    for sample in np.random.randint(1,100,[1,25]):
        x_t.append(x_train[sample])
        y_t.append(y_train[sample])
    w0=np.zeros(shape=(n_feature,1))
    lmd=0.9
    v=0.01
    h=0.01
    wn,train_lossn,validation_lossn=iterationNAG(x_train,x_validation,y_train,y_validation,w0,lmd,v,h)
    plt.plot(validation_lossn, label='validation loss')
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    w1=np.zeros(shape=(n_feature,1))
    h1=0.001
    e=0
    G=0.001
    wr,train_lossr,validation_lossr=iterationRMSProp(x_train,x_validation,y_train,y_validation,w1,lmd,e,h1,G)
    plt.plot(validation_lossr, label='validation loss')
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    w2=np.zeros(shape=(n_feature,1))
    lmd_1=0.95
    dt=0
    wa,train_lossa,validation_lossa=iterationAdaDelta(x_train,x_validation,y_train,y_validation,w2,lmd_1,e,dt,G) 
    plt.plot(validation_lossa, label='validation loss')
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    w3=np.zeros(shape=(n_feature,1))
    lmd_2=0.999
    b=0.9
    m=0
    wam,train_lossam,validation_lossam=iterationAdam(x_train,x_validation,y_train,y_validation,w3,lmd_2,b,m,e,h1,G)
    plt.plot(validation_lossam, label='validation loss')
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()